# Curate DataFrames and AnnDatas

Curating datasets typically means three things:

1. Validate: ensure a dataset meets predefined _validation criteria_
2. Standardize: transform a dataset so that it meets validation criteria, e.g., by fixing typos or using standardized identifiers
3. Annotate: link a dataset against metadata records

In LaminDB, valid metadata is metadata that's stored in a metadata registry and _validation criteria_ merely defines a mapping onto a field of a registry.

```{admonition} Example

`"Experiment 1"` is a valid value for `ULabel.name` if a record with this name exists in the {class}`~lamindb.ULabel` registry.

```

In [ ]:
# !pip install 'lamindb[bionty]'
!lamin init --storage ./test-curate --schema bionty

## Validate a DataFrame

Let's start with a DataFrame that we'd like to validate.

In [ ]:
import lamindb as ln
import bionty as bt
import pandas as pd


df = pd.DataFrame(
    {
        "temperature": [37.2, 36.3, 38.2],
        "cell_type": ["cerebral pyramidal neuron", "astrocyte", "oligodendrocyte"],
        "assay_ontology_id": ["EFO:0008913", "EFO:0008913", "EFO:0008913"],
        "donor": ["D0001", "D0002", "DOOO3"]
    },
    index = ["obs1", "obs2", "obs3"]
)
df

Define validation criteria and create a {class}`~lamindb.Curate` object.

In [ ]:
# in the dictionary, each key is a column name of the dataframe, and each value
# is a registry field onto which values are mapped
categoricals = {
    "cell_type": bt.CellType.name,
    "assay_ontology_id": bt.ExperimentalFactor.ontology_id,
    "donor": ln.ULabel.name,
}

# pass validation criteria
curate = ln.Curate.from_df(df, categoricals=categoricals)

The {meth}`~lamindb.Curate.validate` method checks our data against the defined criteria. It identifies which values are already validated (exist in our registries) and which are potentially problematic (do not yet exist in our registries).

In [ ]:
curate.validate()

## Register new metadata values

If you see "non-validated" values, you'll need to decide whether to add them to your registries or "fix" them in your dataset.

Because our current registries are still empty, here, we'll add new values.

In [ ]:
# this adds cell types that were validated based on a public ontology
curate.add_validated_from("cell_type")

In [ ]:
# use a lookup object to get the correct spelling of categories from a public ontology
lookup = curate.lookup("public")
lookup

In [ ]:
# here is an example for the cell_type column
cell_types = lookup["cell_type"]
cell_types.cerebral_cortex_pyramidal_neuron

In [ ]:
# fix the cell type
df.cell_type = df.cell_type.replace({"cerebral pyramidal neuron": cell_types.cerebral_cortex_pyramidal_neuron.name})
# now register curated and validated cell types
curate.add_validated_from(df.cell_type.name)

In [ ]:
# this adds assays that were validated (via a public ontology)
curate.add_validated_from("assay_ontology_id")

In [ ]:
# this adds donors that were _not_ validated
curate.add_new_from(df.donor.name)

In [ ]:
# validate again
validated = curate.validate()
validated

## Validate an AnnData object

Here we addtionally specify which `var_index` to validate against.

In [ ]:
X = pd.DataFrame(
    {
        "ENSG00000081059": [1, 2, 3], 
        "ENSG00000276977": [4, 5, 6], 
        "ENSG00000198851": [7, 8, 9], 
        "ENSG00000010610": [10, 11, 12], 
        "ENSG00000153563": [13, 14, 15],
        "corrupted": [16, 17, 18]
    }, 
    index=df.index
)

adata = ad.AnnData(X=X, obs=df)
adata

In [ ]:
curate = ln.Curate.from_anndata(
    adata, 
    var_index=bt.Gene.ensembl_gene_id,  # validate var.index against Gene.ensembl_gene_id
    categoricals=categoricals, 
    organism="human",
)

In [ ]:
curate.validate()

## Curate data object to pass validation

Non-validated terms can be accessed via:

In [ ]:
curate.non_validated

Subset anndata object to validated genes only:

In [ ]:
adata_validated = adata[:, ~adata.var.index.isin(curate.non_validated["var_index"])].copy()

Now let's validate the subsetted object:

In [ ]:
curate = ln.Curate.from_anndata(
    adata_validated, 
    var_index=bt.Gene.ensembl_gene_id,  # validate var.index against Gene.ensembl_gene_id
    categoricals=categoricals, 
    organism="human",
)

curate.validate()

## Save a curated artifact

The validated object can be subsequently saved as an {class}`~lamindb.Artifact`:

In [ ]:
artifact = curate.save_artifact(description="test AnnData")

Validated features and labels are linked to the artifact:

In [ ]:
artifact.describe()

We've walked through the process of validating, standardizing, and annotating datasets going through these key steps:

1. Defining validation criteria
2. Validating data against existing registries
3. Adding new validated entries to registries
4. Annotating artifacts with validated metadata

By following these steps, you can ensure your data is standardized and well-curated.

If you have datasets that aren't DataFrame-like or AnnData-like, read: {doc}`curate-any`.